# TomoBase Tutorial

This notebook is designed to show the basic processes of how to use the tomobase library for running tomographic processes. To do so each cell in this library outlines how to perform a procedure with the library:

Table of Contents:
[t](#data-imports)


## Introduction

## 

In [ ]:
import tomobase
from tomobase import data
from tomobase.processes import alignments
import ncempy.io as nio
import numpy as np 
import astra


In [ ]:
from skimage.util import random_noise
import matplotlib.pyplot as plt
from tomobase import phantoms 

vol = tomobase.phantoms.nanocage()
scheme  =  tomobase.tiltschemes.Binary(-64, 64, 8)
angles = np.array([scheme.get_angle() for i in range(65)])
sino = tomobase.processes.project(vol, angles)
sino.data = np.random.poisson(sino.data)

plt.hist(sino.data.ravel(), bins=100, fc='k', ec='k')
plt.show()

plt.imshow(sino.data[0,:, :])

In [ ]:
sino = alignments.align_sinogram_center_of_mass(sino2)
sino = alignments.align_sinogram_xcorr(sino)
sino = alignments.background_subtract_median(sino)
sino.show()

In [ ]:
sino = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpinc_aligned.mrc')
obj = alignments.MaskBackgroundManual(sino)


In [ ]:
from skimage.filters import threshold_otsu
from scipy.ndimage import  binary_dilation
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import tomobase.processes.alignments

sino = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpinc_aligned.mrc')
threshold = threshold_otsu(sino.data)
mask = sino.data < threshold
mask = binary_dilation(mask, iterations=4)

sino_inc = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpinc_aligned.mrc')
sino_bin =  tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpbin_aligned.mrc')
sino_grs = tomobase.data.Sinogram.from_file(r'\\ematbyname\emat\TimC\tiltscheme_paper\aunpgrs_aligned.mrc')

sinos = [sino_inc, sino_bin, sino_grs]
for sino in sinos:
    threshold = threshold_otsu(sino.data)
    mask = sino.data < threshold
    mask = binary_dilation(mask, iterations=4)
    sino.data[~mask] = 0
    sino = tomobase.processes.alignments.normalize(sino)
    rec = tomobase.processes.reconstruct_weighted_sirt(sino, iterations=100, weighted=True)
    sino = tomobase.processes.alignments.weight_by_angle(sino)
    
    projections = tomobase.processes.project(rec, sino.angles)
    mae = np.mean(np.abs(projections.data - sino.data))
    ssim = structural_similarity(projections.data, sino.data, data_range=(sino.data.max() - sino.data.min()))
    print(mae, ssim)
    



In [ ]:
from tomobase import processes
import tomobase.processes.alignments

sino = tomobase.processes.alignments.align_tilt_axis_shift(sino)

In [ ]:
sino2 = obj.sino  
sino.show()

In [ ]:
sino = processes.alignments.bin(sino)
rec = processes.reconstruct(sino, 'sirt', iterations=100)
rec.to_file('AuNP-Incremental.rec')